In [1]:
import os,textwrap
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown,display
from dotenv import load_dotenv
from openai import OpenAI
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
assert api_key and len(api_key)> 20,"Missing/Invalid OPEN_API_KEY"
client = OpenAI(api_key = api_key)


In [2]:
class Website:
    def __init__(self, url : str):
        self.url = url
        r = requests.get(url,timeout = 20)
        r.raise_for_status()
        soup = BeautifulSoup(r.text,"html.parser")
        for tag in soup(["script", "style", "noscript", "img", "input"]):
            tag.decompose()

        self.title = (soup.title.string or "").strip() if soup.title else ""
        self.text = " ".join(soup.get_text(" ").split())

def summarize(website: Website, bullets: int = 6):
    prompt = f"""
    You are a helpful assistant. Summarize the page at {website.url}.
    Return Markdown with a short intro + {bullets} bullet highlights.
    If sections are obvious, include headings. Keep it concise for beginners.
    Page title: {website.title or "N/A"}
    Page text:
    {website.text[:12000]}  # safety truncate
    """
    resp = client.chat.completions.create(
        model="gpt-4o-mini",  # lightweight GPT-4-class model; name may vary
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2,
    )
    return resp.choices[0].message.content

def show_markdown(md: str):
    display(Markdown(md))


In [ ]:

site = Website("https://www.w3schools.com/")
md = summarize(site)
show_markdown(md)
